In [43]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import model_selection, metrics, datasets, naive_bayes, preprocessing, pipeline
import matplotlib.pyplot as plt
from matplotlib import colors, patches
from IPython.core.interactiveshell import InteractiveShell


In [2]:
# 配置项
%config IPCompleter.greedy=True

# 这个要放到设置中文之前否则还是小方框
plt.style.use("seaborn")

# 指定默认字体 用来正常显示中文标签
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决保存图像是负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

# #全部行都能输出
InteractiveShell.ast_node_interactivity = "all"

### 思路:求解argmaxc P(c|w) = argmaxc P(w|c) * P(c) / P(w)
- P(c) 文章中单词被拼写正确的概率
- P(w|c) 是已知正确的单词c的前提下,被拼错成w单词的概率
- argmaxc, 用来枚举所有可能的 c 并且选取概率最大的


In [21]:
def words(text):
    """
    将所有单词全部提前出来,并且剔除特殊字符
    """
#     flag 支持re.I 不区分大小写
#     re.I(re.IGNORECASE) 使匹配对大小写不敏感
#     re.L(re.LOCAL) 做本地化识别（locale-aware）匹配
#     re.M(re.MULTILINE) 多行匹配，影响 ^ 和 $
#     re.S(re.DOTALL) 使 . 匹配包括换行在内的所有字符
#     re.U(re.UNICODE) 根据Unicode字符集解析字符。这个标志影响 \w, \W, \b, \B.
#     re.X(re.VERBOSE) 该标志通过给予你更灵活的格式以便你将正则表达式写得更易于理解。
    return re.findall('[a-z]+', text.lower(), flags=0)

def train(features):
    """
    统计单词出现的频次
    要是遇到我们从来没有过见过的新词怎么办. 假如说一个词拼写完全正确, 但是语料库中没有包含这个词, 
    从而这个词也永远不会出现在训练集中. 于是, 我们就要返回出现这个词的概率是0. 这个情况不太妙, 
    因为概率为0这个代表了这个事件绝对不可能发生, 而在我们的概率模型中, 
    我们期望用一个很小的概率来代表这种情况. lambda: 1
    """
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

In [22]:
# 读取单词
NWORDS = train(words(open('big.txt').read()))
 
alphabet = 'abcdefghijklmnopqrstuvwxyz'

### 编辑距离
两个词之间的编辑距离定义为了使用插入(在词中插入一个字母),删除(从词中删除一个字母),
交换(交换相邻的两个字母),替换(将一个字母天幻成另外一个字母)的操作从一个词变成另外一个词

In [28]:
# 返回所有与单词w编辑距离为1的集合
def edits1(word):
    n = len(word)
    # 在单词中插入一个字母
    insert_words = [word[:i] + c + word[i:] for i in range(n+1) for c in alphabet]
    # 从单词删除一个字母
    delete_words = [word[:i] + word[i+1:] for i in range(n)]
    # 交换某一个字符
    change_words = [word[:i] + word[i+1] + word[i] + word[i+2:] for i in range(n-1)]
    # 替换其中的某一个字符
    replace_words = [word[:i] + c + word[i+1:] for i in range(n) for c in alphabet]
    # 去重返回
    return set(insert_words + delete_words + change_words + replace_words)

In [36]:
# 返回所有与候选单词编辑距离为2的单词 这样写比我想象中的高明
def edits2(word):
    return set(w2 for w1 in edits1(word) for w2 in edits1(w1))

# edits2改进版 只返回资料库中存在的单词
def known_edits2(word):
    return set(w2 for w1 in edits1(word) for w2 in edits1(w1) if w2 in NWORDS)

In [33]:
len(edits2('something'))

114324

In [37]:
# 识别单词,把正确的单词当做候选词
def known(words):
    return set(w for w in words if w in NWORDS)

#如果known(set)非空, candidate 就会选取这个集合, 而不继续计算后面的
def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=lambda w: NWORDS[w])

In [42]:
# 验证检查
correct('knon')
correct('appl')
correct('apple')
correct('tess')

'know'

'apply'

'apple'

'less'